In [2]:
import os
import openai
import numpy as np
import pandas as pd
from openai import OpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = "XXXX"
client = OpenAI()
print(client.models.list())

SyncPage[Model](data=[Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created

In [5]:
exp_sen = "Rocky & Rani ke prem kayani is zabardast"

embed = client.embeddings.create(
    model="text-embedding-ada-002",
    input=exp_sen,
)

embed.data[0].embedding[:10]

[-0.0042231022380292416,
 -0.027315940707921982,
 -0.004930226132273674,
 -0.01668289117515087,
 -0.033680059015750885,
 0.029751591384410858,
 -0.01466627698391676,
 -0.04326551780104637,
 -0.01233538705855608,
 -0.026648102328181267]

In [5]:
toy_dataset = [
    "Rocky & Rani ke prem kayani is zabardast love story",
    "Bhaag Milka Bhaag is all about sports enthu",
    "Stree is a really horror scary movie"
]

In [8]:
embeddings = client.embeddings.create(
    model="text-embedding-ada-002",
    input= toy_dataset,
)

pure_embeds = []
for embedding in embeddings.data:
    pure_embeds.append(embedding.embedding)

[[-0.0047816261649131775, -0.03443290665745735, -0.004044242203235626, -0.015852130949497223, -0.030197009444236755, 0.03206808120012283, -0.00629862304776907, -0.045347485691308975, -0.017294414341449738, -0.022491833195090294, 0.002934918040409684, 0.017775176092982292, 0.0009509653900749981, -0.00931637454777956, 0.004875829443335533, 0.009751658886671066, 0.028715744614601135, -0.02351832389831543, 0.011512285098433495, -0.0012749920133501291, -0.0009558379533700645, 0.003933797124773264, 0.007620716467499733, -0.020945601165294647, 0.0009087363723665476, 0.02470073662698269, 0.02739040181040764, -0.027780208736658096, 0.0039240517653524876, -0.010576749220490456, -0.01633289083838463, 0.009764652699232101, 0.0014512169873341918, -0.009179942309856415, -0.022738711908459663, -0.016280917450785637, -0.004200164694339037, 0.005570984445512295, 0.002267374424263835, 0.010355859063565731, 0.012142471969127655, 0.0026214485988020897, -0.009732168167829514, -0.01212947815656662, -0.01481

In [19]:
user_request = input("what movie you are looking for?")

user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input= user_request,
)

what movie you are looking for? Sports


In [22]:
user_vector = user_vector.data[0].embedding

#Normalize the User Vector
user_vector_norm = user_vector / np.linalg.norm(user_vector)

pure_embeds_norm = pure_embeds / np.linalg.norm(pure_embeds,axis=1,keepdims=True)

cosine_similarity_scores = np.dot(user_vector_norm,pure_embeds_norm.T)
print(cosine_similarity_scores)

[0.73788084 0.81902838 0.75489283]


In [15]:
from scipy import spatial

result = 1- spatial.distance.cosine(user_vector,pure_embeds[0])
print(result)

0.8120288149833951


In [23]:
sorted_indices = np.argsort(1-cosine_similarity_scores)
priority_values  = [toy_dataset[index] for index in sorted_indices]

print("User query :",user_request)
print("Recommended movies: ")
for i in range(len(priority_values)):
    movie = priority_values[i]
    print(i+1,":",movie)

User query : Sports
Recommended movies: 
1 : Bhaag Milka Bhaag is all about sports enthu
2 : Stree is a really horror scary movie
3 : Rocky & Rani ke prem kayani is zabardast love story


In [31]:
dataset = pd.read_csv("C:/Users/prohr/Downloads/movies_metadata.csv")

C:\Users\prohr\AppData\Local\Temp\ipykernel_372\3774853388.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("C:/Users/prohr/Downloads/movies_metadata.csv")


In [33]:
dataset.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [34]:
subset = dataset[['title','overview']]
subset.head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [35]:
subset.shape

(45466, 2)

In [36]:
small_dataset = subset.iloc[:100]
small_dataset.shape

(100, 2)

In [37]:
small_dataset.dropna(inplace=True)
small_dataset.shape

C:\Users\prohr\AppData\Local\Temp\ipykernel_372\1420628433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_dataset.dropna(inplace=True)


(99, 2)

In [40]:
small_embeds = client.embeddings.create(
    model="text-embedding-ada-002",
    input=small_dataset['overview'].values.tolist()
)

In [41]:
small_vectors = []
for embedding in small_embeds.data:
    small_vectors.append(embedding.embedding)

small_vectors[1][:10]

[0.01767769642174244,
 -0.03540753945708275,
 -0.016100261360406876,
 -0.012326152063906193,
 -0.0070593454875051975,
 0.02247518301010132,
 -0.02235785312950611,
 -0.022449109703302383,
 -0.013662408106029034,
 -0.011055078357458115]

In [44]:
user_request = input("What movie are you looking for? ")

user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input=user_request)

user_vector = user_vector.data[0].embedding

# Normalize the user_vector
user_vector_norm = user_vector / np.linalg.norm(user_vector)

small_vector_norm = small_vectors / np.linalg.norm(small_vectors,axis=1, keepdims=True)

cosine_similarity_score = np.dot(user_vector_norm,small_vector_norm.T)

What movie are you looking for?  Vacation


In [45]:
sorted_indices = np.argsort(-cosine_similarity_score)

prioritized_movies = [small_dataset.iloc[index] for index in sorted_indices]

print("User's query: ", user_request)
print("Recommended movies: ")
for i in range(10):
    movie = prioritized_movies[i]
    print(i+1,":",movie['title'])

User's query:  Vacation
Recommended movies: 
1 : Home for the Holidays
2 : White Squall
3 : Friday
4 : Kicking and Screaming
5 : Dunston Checks In
6 : Last Summer in the Hamptons
7 : Beautiful Girls
8 : When Night Is Falling
9 : Across the Sea of Time
10 : Cry, the Beloved Country


In [46]:
pip install pinecone-client

   ---------------------------------------- 0.0/216.2 kB ? eta -:--:--
   ---------------------------------------- 216.2/216.2 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [47]:
from pinecone import Pinecone

pc = Pinecone(api_key="ABCD")
index = pc.Index("mmm")

In [48]:
for i in range(len(small_dataset)):
    upsert_response = index.upsert(
    vectors=[
        (
         str(i),
         small_vectors[i],
         {"title": small_dataset.iloc[i]['title']}
        )
    ])

In [62]:
user_request = input("What movie are you looking for? ")

user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input=user_request)

user_vector = user_vector.data[0].embedding

What movie are you looking for?  Holiday


In [64]:
matches = index.query(
    vector=user_vector,
    top_k=10,
    include_metadata=True)

matches

{'matches': [{'id': '55',
              'metadata': {'title': 'Home for the Holidays'},
              'score': 0.805510581,
              'values': []},
             {'id': '93',
              'metadata': {'title': "A Midwinter's Tale"},
              'score': 0.790248752,
              'values': []},
             {'id': '91',
              'metadata': {'title': 'Beautiful Girls'},
              'score': 0.789563,
              'values': []},
             {'id': '67',
              'metadata': {'title': 'Friday'},
              'score': 0.788457513,
              'values': []},
             {'id': '52',
              'metadata': {'title': 'The Big Green'},
              'score': 0.786545277,
              'values': []},
             {'id': '82',
              'metadata': {'title': 'Last Summer in the Hamptons'},
              'score': 0.785669744,
              'values': []},
             {'id': '58',
              'metadata': {'title': 'The Indian in the Cupboard'},
              'sco